# Battle of the Neighborhoods

This project will analyze neighborhoods between Toronto, Canada and New York City, New York. A Fortune 500 company is looking to move its headquarters to either Toronto or New York City. The company wants insight into the neighborhoods and local businesses in the cities so that its employees may have the optimum living standards and quality of life. This project will explore the similarities and dissimilarities between certain neighborhoods in the two cities, and determine which neighborhoods best fit the culture of the Fortune 500 company’s employees.

## Segmenting and Clustering Neighborhoods in Toronto

### Use data from Toronto Neighborhood Wikipedia page to segment, cluster and explore neighborhoods in Toronto

In [52]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim #convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

#print('Libraries imported.')

## todo esto fue importado

#import requests # library to handle requests
#import pandas as pd # library for data analsysis
#import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
#from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Download and Explore Toronto Dataset
The dateset being used is found at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

The dataset is a list of Toronto's zipcodes which includes the boroughs and neighborhood names.

In [14]:
#Obtain Postal Code, Borough, and Neighborhood information from Wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#Obtain the first table
df_toronto = table[0]
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Transform the data

In [15]:
df_toronto.rename(columns = {"Postal Code": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)
#df.head()

#Combine the neighborhoods that exists in one postal code
df_toronto = df_toronto.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
#df.head()

#Change unassigned Neighborhood to its Borough's name
df_toronto.loc[85,'Neighborhood'] = 'Queen\'s Park'

print (df_toronto.shape)

df_toronto.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Join neighborhood table with latitude and longitude information

In [16]:
#Create a dataframe of the latitude and longitudes of the Toronto Neighborhoods
latlong = pd.read_csv("http://cocl.us/Geospatial_data")
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
latlong.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
latlong.shape

(103, 3)

In [19]:
latlong.tail()

,PostalCode,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


### Join latitude and longitude dataframe with neighborhood dataframe

In [20]:
#Join the Lat and Long dataframe to Neighborhoods dataframe
df_toronto.set_index("PostalCode")
latlong.set_index("PostalCode")
neighbor=pd.merge(df_toronto, latlong)
neighbor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [21]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(neighbor['Borough'].unique()),
        neighbor.shape[0]
    )
)

Toronto has 10 boroughs and 103 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto, Canada.

In [23]:
CLIENT_ID = '3WAK1JKOYXAXBXADAY1MX12TJF3CHBO5EK5A1QQK0MG5GRDX' # your Foursquare ID
CLIENT_SECRET = '5GTPEHWTNVGIVO3BJNKKKV22WJDDKIMHIURZAFO3SQYWAC5U' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



address = 'Toronto, CA'

#geolocator = Nominatim()
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

Your credentails:
CLIENT_ID: 3WAK1JKOYXAXBXADAY1MX12TJF3CHBO5EK5A1QQK0MG5GRDX
CLIENT_SECRET:5GTPEHWTNVGIVO3BJNKKKV22WJDDKIMHIURZAFO3SQYWAC5U
The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top.

In [24]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Scarborough Neighborhood data.

In [25]:
scarborough_data = neighbor[neighbor['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [26]:
scarborough_data.shape

(17, 5)

Lets get the geographical coordinates of Scarborough

In [28]:
address = 'Scarborough, Toronto'

#geolocator = Nominatim()
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough, CA are 43.773077, -79.257774.


In [29]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### Explore Scarborough neighborhood in Toronto with Foursquare API
#### Define Foursquare credentials and version

In [31]:
CLIENT_ID = '3WAK1JKOYXAXBXADAY1MX12TJF3CHBO5EK5A1QQK0MG5GRDX' # your Foursquare ID
CLIENT_SECRET = '5GTPEHWTNVGIVO3BJNKKKV22WJDDKIMHIURZAFO3SQYWAC5U' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3WAK1JKOYXAXBXADAY1MX12TJF3CHBO5EK5A1QQK0MG5GRDX
CLIENT_SECRET:5GTPEHWTNVGIVO3BJNKKKV22WJDDKIMHIURZAFO3SQYWAC5U


In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [32]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Found 89 venues in 17 neighborhoods.


In [33]:
print(scarborough_venues.shape)
scarborough_venues.head()

(89, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [34]:
#Venues per Neighborhood
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [35]:
print('There are {} distinct venues in {} categories.'.format(
    len(scarborough_venues['Venue'].unique()),len(scarborough_venues['Venue Category'].unique())))

#print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 80 distinct venues in 55 categories.


### Analyze each Neighborhood


In [36]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.00,0.000,0.000000,0.2,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.2,0.2,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.20,0.000000,0.0,0.000000,0.00,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.25,0.000,0.000000,0.0,0.000000,0.25,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.25,0.000000,0.0,0.000000,0.00,0.0
2,Cedarbrae,0.000000,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0,0.0,0.00,0.125,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.125000,0.125000,0.00,0.000000,0.000000,0.125,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.125000,0.00,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.0,0.000,0.000000,0.000000,0.083333,0.0,0.000000,0.0,0.0,0.00,0.000,0.083333,0.0,0.000000,0.00,0.0,0.00,0.000,0.083333,0.083333,0.083333,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.083333,0.083333,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.083333,0.000000,0.0,0.083333,0.166667,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.083333,0.00,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.000000,0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.5,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.200000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.400000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.1,0.000000,0.100000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.1,0.000,0.000000,0.0,0.000000,0.100000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.1,0.000000,0.00,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.0,0.000,0.000000,0.000000,0.125000,0.0,0.125000,0.0,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.125,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.125000,0.000000,0.000000,0.00,0.0,0.0,0.125,0.0,0.125,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.125,0.125,0.

### Each neighborhood with the top 5 venues

In [38]:
num_top_venues = 10

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2               Skating Rink   0.2
3             Breakfast Spot   0.2
4             Clothing Store   0.2
5          Accessories Store   0.0
6                       Park   0.0
7              Jewelry Store   0.0
8          Korean Restaurant   0.0
9             Medical Center   0.0


----Birch Cliff, Cliffside West----
                       venue  freq
0               Skating Rink  0.25
1      General Entertainment  0.25
2                       Café  0.25
3            College Stadium  0.25
4          Accessories Store  0.00
5                       Park  0.00
6              Jewelry Store  0.00
7          Korean Restaurant  0.00
8  Latin American Restaurant  0.00
9                     Lounge  0.00


----Cedarbrae----
                  venue  freq
0    Athletics & Sports  0.12
1   Fried Chicken Joint  0.12
2                Bakery  0.12
3                  Bank  0.12
4   

### Let's put that into a pandas dataframe
#### Let's write a function to sort the venues in descending order

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    

Create a new dataframe and display the top ten venues for each neighborhood

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,None,None,None,None,None,None,None,None,None,None
1,"Birch Cliff, Cliffside West",None,None,None,None,None,None,None,None,None,None
2,Cedarbrae,None,None,None,None,None,None,None,None,None,None
3,"Clarks Corners, Tam O'Shanter, Sullivan",None,None,None,None,None,None,None,None,None,None
4,"Cliffside, Cliffcrest, Scarborough Village West",None,None,None,None,None,None,None,None,None,None
5,"Dorset Park, Wexford Heights, Scarborough Town...",None,None,None,None,None,None,None,None,None,None
6,"Golden Mile, Clairlea, Oakridge",None,None,None,None,None,None,None,None,None,None
7,"Guildwood, Morningside, West Hill",None,None,None,None,None,None,None,None,None,None
8,"Kennedy Park, Ionview, East Birchmount Park",None,None,None,None,None,None,None,None,None,None
9,"Malvern, Rouge",None,None,None,None,None,None,None,None,None,None


In [41]:
neighborhoods_venues_sorted.iloc[11,:]

Neighborhood              Rouge Hill, Port Union, Highland Creek
1st Most Common Venue                                       None
2nd Most Common Venue                                       None
3rd Most Common Venue                                       None
4th Most Common Venue                                       None
5th Most Common Venue                                       None
6th Most Common Venue                                       None
7th Most Common Venue                                       None
8th Most Common Venue                                       None
9th Most Common Venue                                       None
10th Most Common Venue                                      None
Name: 11, dtype: object

### 4. Cluster the Scarborough Neighborhood using k-means
Run K-means to cluster neighborhood into three clusters

In [42]:
# set number of clusters
kclusters = 3

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

Lets create a new dataframe that includes the cluster as well as the top ten venues for each neighborhood

In [43]:
#Note that the neighborhood Upper Rouge does not have any venues, so I will drop from dataset
scarborough_data.drop(scarborough_data[scarborough_data.Neighborhood == 'Upper Rouge'].index, inplace = True)
#df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)

scarborough_merged = scarborough_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,None,None,None,None,None,None,None,None,None,None
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,None,None,None,None,None,None,None,None,None,None
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,None,None,None,None,None,None,None,None,None,None
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,None,None,None,None,None,None,None,None,None,None
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,None,None,None,None,None,None,None,None,None,None


In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5. Examine Scarborough Neighborhood Cluster
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

Scarborough Clusters 0, 1, 2

In [45]:
scarborough_cluster_0 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_1 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_2 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

In [46]:
scarborough_cluster_0

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,0,None,None,None,None,None,None,None,None,None,None
1,Scarborough,-79.160497,0,None,None,None,None,None,None,None,None,None,None
2,Scarborough,-79.188711,0,None,None,None,None,None,None,None,None,None,None
3,Scarborough,-79.216917,0,None,None,None,None,None,None,None,None,None,None
5,Scarborough,-79.239476,0,None,None,None,None,None,None,None,None,None,None
6,Scarborough,-79.262029,0,None,None,None,None,None,None,None,None,None,None
7,Scarborough,-79.284577,0,None,None,None,None,None,None,None,None,None,None
8,Scarborough,-79.239476,0,None,None,None,None,None,None,None,None,None,None
10,Scarborough,-79.273304,0,None,None,None,None,None,None,None,None,None,None
11,Scarborough,-79.295849,0,None,None,None,None,None,None,None,None,None,None


In [47]:
scarborough_cluster_1

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,-79.239476,1,None,None,None,None,None,None,None,None,None,None


In [48]:
scarborough_cluster_2

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,-79.264848,2,None,None,None,None,None,None,None,None,None,None


# 6. Explore New York City Neighborhoods
New York City has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. The link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a wget command and access the data. So let's go ahead and do that.

In [65]:
!brew install wget

#!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
#$ conda update -n base -c defaults conda

!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
#!curl -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

#!wget -q -O 'newyork_data.json' https://raw.githubusercontent.com/jkgiesler/parse-chicago-neighborhoods/master/Neighborhoods_2012_polygons.json
print('Data downloaded!')

Updating Homebrew...
To reinstall 1.20.3_2, run `brew reinstall wget`
Data downloaded!


In [66]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [67]:
neighborhoods_data = newyork_data['features']

In [68]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

## Transform the data into a pandas dataframe

In [63]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [16]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [17]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [18]:
neighborhoods.shape

(306, 4)

In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Use Geolibrary to get the latitude and longitude of New York City

In [21]:
address = 'New York City, NY'

#geolocator = Nominatim(user_agent = 'my-application')
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [22]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Lets simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Queens neighborhood data.

In [23]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


Lets get the geographical location of Queens, NY

In [25]:
address = 'Queens, NY'

#geolocator = Nominatim(user_agent = 'my-application')
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [51]:
# create map of Manhattan using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

### Lets explore the Long Island City neighborhood in Queens, NY

In [27]:
queens_data.loc[10, 'Neighborhood']

'Long Island City'

In [28]:
#Long Island City Latitude and Longitude values

neighborhood_latitude = queens_data.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = queens_data.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = queens_data.loc[10, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Long Island City are 40.75021734610528, -73.93920223915505.


## Top 100 venues in Long Island City neighborhood within a radius of 500 meters
First, let's create the GET request URL named url.

In [32]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=3WAK1JKOYXAXBXADAY1MX12TJF3CHBO5EK5A1QQK0MG5GRDX&client_secret=5GTPEHWTNVGIVO3BJNKKKV22WJDDKIMHIURZAFO3SQYWAC5U&v=20180604&ll=40.75021734610528,-73.93920223915505&radius=500&limit=100'

In [33]:
#Send the GET request
results = requests.get(url).json()

In [34]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/Reggie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Chip,Dessert Shop,40.750069,-73.940831
1,Hilton Garden Inn,Hotel,40.750216,-73.936886
2,Etto Espresso Bar,Coffee Shop,40.748703,-73.940689
3,Dutch Kills,Cocktail Bar,40.747830,-73.940108
4,Brooklyn Boulders Queensbridge,Climbing Gym,40.752649,-73.940010


In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

71 venues were returned by Foursquare.


## 7. Analyze Each Neighborhood in Queens

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [38]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude'])

Found 2102 venues in 81 neighborhoods.


In [39]:
queens_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
2,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
3,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant


In [40]:
print(queens_venues.shape)
queens_venues.tail()

(2102, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2097,Queensbridge,40.756091,-73.945631,Queensbridge Basketball Courts,40.755060,-73.949103,Basketball Court
2098,Queensbridge,40.756091,-73.945631,The Ravel Hotel Gym,40.753787,-73.948815,Athletics & Sports
2099,Queensbridge,40.756091,-73.945631,Profundo Pool Club,40.753719,-73.948878,Hotel Pool
2100,Queensbridge,40.756091,-73.945631,Estate Garden And Grill,40.753700,-73.948841,Beer Garden
2101,Queensbridge,40.756091,-73.945631,Track 114,40.753008,-73.947833,Platform


In [41]:
#Venues per Neighborhood
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,20,20,20,20,20,20
Astoria,99,99,99,99,99,99
Astoria Heights,13,13,13,13,13,13
Auburndale,21,21,21,21,21,21
Bay Terrace,37,37,37,37,37,37
Bayside,72,72,72,72,72,72
Bayswater,1,1,1,1,1,1
Beechhurst,16,16,16,16,16,16
Bellaire,13,13,13,13,13,13


In [42]:
print('There are {} distinct venues in {} categories.'.format(
    len(queens_venues['Venue'].unique()),len(queens_venues['Venue Category'].unique())))

There are 1736 distinct venues in 277 categories.


In [43]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
#scarborough_onehot = scarborough_onehot[fixed_columns]

neighbor = queens_onehot['Neighborhood']
queens_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
queens_onehot.insert(0, 'Neighborhood', neighbor)

queens_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,W

### Group by Neighborhood, and exame the frequency of the occurrence of venue

In [44]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,W

Each Neighborhood with the top 5 venues

In [45]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
            venue  freq
0       Surf Spot  0.20
1   Metro Station  0.10
2  Sandwich Place  0.10
3           Beach  0.05
4     Pizza Place  0.05


----Astoria----
                       venue  freq
0                        Bar  0.06
1  Middle Eastern Restaurant  0.06
2                 Hookah Bar  0.05
3                Pizza Place  0.04
4         Seafood Restaurant  0.04


----Astoria Heights----
                venue  freq
0  Italian Restaurant  0.08
1          Laundromat  0.08
2        Burger Joint  0.08
3         Supermarket  0.08
4       Bowling Alley  0.08


----Auburndale----
                  venue  freq
0    Italian Restaurant  0.10
1             Pet Store  0.05
2    Miscellaneous Shop  0.05
3        Mattress Store  0.05
4  Fast Food Restaurant  0.05


----Bay Terrace----
               venue  freq
0     Clothing Store  0.11
1         Shoe Store  0.08
2     Cosmetics Shop  0.05
3         Donut Shop  0.05
4  Mobile Phone Shop  0.05


----Bayside----
               

4  Chinese Restaurant  0.06


----St. Albans----
                  venue  freq
0  Caribbean Restaurant  0.21
1            Donut Shop  0.05
2     Convenience Store  0.05
3                  Café  0.05
4   Fried Chicken Joint  0.05


----Steinway----
                 venue  freq
0  Rental Car Location  0.12
1       Cosmetics Shop  0.08
2        Deli / Bodega  0.08
3         Dessert Shop  0.04
4           Bagel Shop  0.04


----Sunnyside----
                       venue  freq
0                Pizza Place  0.12
1         Chinese Restaurant  0.07
2                     Bakery  0.07
3  South American Restaurant  0.05
4              Deli / Bodega  0.05


----Sunnyside Gardens----
                 venue  freq
0                  Bar  0.07
1          Pizza Place  0.04
2        Grocery Store  0.04
3          Coffee Shop  0.03
4  American Restaurant  0.03


----Utopia----
             venue  freq
0  Automotive Shop  0.11
1    Deli / Bodega  0.11
2       Playground  0.06
3   Ice Cream Shop  0.06
4   

In [46]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top venues for each neighborhood

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Metro Station,Sandwich Place,Playground,Restaurant,Bus Stop,Café,Board Shop,Beach,Thai Restaurant
1,Astoria,Middle Eastern Restaurant,Bar,Hookah Bar,Seafood Restaurant,Pizza Place,Mediterranean Restaurant,Café,Bakery,Indian Restaurant,Greek Restaurant
2,Astoria Heights,Plaza,Bakery,Burger Joint,Food,Supermarket,Bowling Alley,Chinese Restaurant,Laundromat,Bus Station,Hostel
3,Auburndale,Italian Restaurant,Comic Shop,Athletics & Sports,Supermarket,Fast Food Restaurant,Mattress Store,Korean Restaurant,Bar,Gymnastics Gym,Toy / Game Store
4,Bay Terrace,Clothing Store,Shoe Store,Women's Store,Donut Shop,Mobile Phone Shop,Kids Store,American Restaurant,Cosmetics Shop,Coffee Shop,Movie Theater
5,Bayside,Bar,Pizza Place,Indian Restaurant,Chinese Restaurant,Sushi Restaurant,Spa,Italian Restaurant,Burger Joint,Bakery,Ice Cream Shop
6,Bayswater,Playground,Yoga Studio,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market
7,Beechhurst,Chinese Restaurant,Italian Restaurant,Shopping Mall,Supermarket,Shopping Plaza,Park,Donut Shop,Gym,Boutique,Optical Shop
8,Bellaire,Deli / Bodega,Breakfast Spot,Grocery Store,Gym,Bus Station,Coffee Shop,Halal Restaurant,Chinese Restaurant,Greek Restaurant,Convenience Store
9,Belle Harbor,Beach,Deli / Bodega,Pub,Spa,Italian Restaurant,Chinese Restaurant,Pharmacy,Mexican Restaurant,Donut Shop,Boutique


In [48]:
neighborhoods_venues_sorted.iloc[47,]

Neighborhood                  Long Island City
1st Most Common Venue                    Hotel
2nd Most Common Venue              Coffee Shop
3rd Most Common Venue              Pizza Place
4th Most Common Venue                      Bar
5th Most Common Venue                     Café
6th Most Common Venue     Gym / Fitness Center
7th Most Common Venue       Mexican Restaurant
8th Most Common Venue               Donut Shop
9th Most Common Venue              Bus Station
10th Most Common Venue             Cheese Shop
Name: 47, dtype: object

## Cluster the Queens Borough using K-Means

In [49]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 4, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2], dtype=int32)

### Dataframe that includes the cluster of each neighborhood

In [50]:
queens_merged = queens_data

# add clustering labels
queens_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,2,Middle Eastern Restaurant,Bar,Hookah Bar,Seafood Restaurant,Pizza Place,Mediterranean Restaurant,Café,Bakery,Indian Restaurant,Greek Restaurant
1,Queens,Woodside,40.746349,-73.901842,2,Grocery Store,Latin American Restaurant,Thai Restaurant,Filipino Restaurant,Bakery,Donut Shop,Pub,American Restaurant,Bar,Pizza Place
2,Queens,Jackson Heights,40.751981,-73.882821,2,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Mexican Restaurant,Thai Restaurant,Grocery Store,Pizza Place,Shoe Store
3,Queens,Elmhurst,40.744049,-73.881656,2,Thai Restaurant,Mexican Restaurant,Indonesian Restaurant,Vietnamese Restaurant,Chinese Restaurant,South American Restaurant,Bubble Tea Shop,Argentinian Restaurant,Bar,Park
4,Queens,Howard Beach,40.654225,-73.838138,2,Pharmacy,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Deli / Bodega,Clothing Store,Gym,Tapas Restaurant,Shipping Store,Sushi Restaurant


In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Queens Cluster Neighborhood
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

In [54]:
queens_cluster_0 = queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_1 = queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_2 = queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_3 = queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_4 = queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

In [55]:
queens_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Flushing,0,Hotpot Restaurant,Korean Restaurant,Chinese Restaurant,Bubble Tea Shop,Construction & Landscaping,Karaoke Bar,Bakery,Asian Restaurant,Café,Gym
10,Long Island City,0,Hotel,Coffee Shop,Pizza Place,Bar,Café,Gym / Fitness Center,Mexican Restaurant,Donut Shop,Bus Station,Cheese Shop
12,East Elmhurst,0,Donut Shop,Indie Movie Theater,Latin American Restaurant,Lake,Supermarket,Gas Station,Rental Car Location,Bus Station,Hotel Bar,Ice Cream Shop
13,Maspeth,0,Pizza Place,Diner,Mobile Phone Shop,Chinese Restaurant,Grocery Store,Bank,Deli / Bodega,Bakery,Sports Bar,Sandwich Place
14,Ridgewood,0,Pizza Place,Bank,Bakery,Mobile Phone Shop,Greek Restaurant,Grocery Store,Café,Mexican Restaurant,Sushi Restaurant,Supplement Shop
15,Glendale,0,Deli / Bodega,Food & Drink Shop,Pizza Place,Arts & Crafts Store,Brewery,Food Stand,Food Court,Food Truck,Food,Eye Doctor
30,Briarwood,0,Deli / Bodega,Playground,Bus Station,Gym,Coffee Shop,Home Service,Indian Restaurant,Convenience Store,Discount Store,Falafel Restaurant
31,Jamaica Center,0,Pizza Place,Mobile Phone Shop,Clothing Store,Department Store,Performing Arts Venue,Coffee Shop,Sandwich Place,Caribbean Restaurant,Mexican Restaurant,Food Court
53,Holliswood,0,Fried Chicken Joint,Mobile Phone Shop,Playground,Grocery Store,Sandwich Place,Donut Shop,Yoga Studio,Filipino Restaurant,Falafel Restaurant,Farm
68,Jamaica Hills,0,Indian Restaurant,Fast Food Restaurant,Pharmacy,Donut Shop,Fried Chicken Joint,Breakfast Spot,Southern / Soul Food Restaurant,Bus Station,Shopping Mall,Chinese Restaurant


In [56]:
queens_cluster_1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Forest Hills,1,Gym / Fitness Center,Gym,Yoga Studio,Pizza Place,Convenience Store,Park,Thai Restaurant,Pharmacy,Optical Shop,Chinese Restaurant


In [57]:
queens_cluster_2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,2,Middle Eastern Restaurant,Bar,Hookah Bar,Seafood Restaurant,Pizza Place,Mediterranean Restaurant,Café,Bakery,Indian Restaurant,Greek Restaurant
1,Woodside,2,Grocery Store,Latin American Restaurant,Thai Restaurant,Filipino Restaurant,Bakery,Donut Shop,Pub,American Restaurant,Bar,Pizza Place
2,Jackson Heights,2,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Mexican Restaurant,Thai Restaurant,Grocery Store,Pizza Place,Shoe Store
3,Elmhurst,2,Thai Restaurant,Mexican Restaurant,Indonesian Restaurant,Vietnamese Restaurant,Chinese Restaurant,South American Restaurant,Bubble Tea Shop,Argentinian Restaurant,Bar,Park
4,Howard Beach,2,Pharmacy,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Deli / Bodega,Clothing Store,Gym,Tapas Restaurant,Shipping Store,Sushi Restaurant
5,Corona,2,Mexican Restaurant,Supermarket,Ice Cream Shop,Park,Deli / Bodega,Chinese Restaurant,Empanada Restaurant,Sandwich Place,South American Restaurant,Bakery
7,Kew Gardens,2,Chinese Restaurant,Indian Restaurant,Bar,Bank,Donut Shop,Cosmetics Shop,Pizza Place,Park,Pet Store,Deli / Bodega
8,Richmond Hill,2,Latin American Restaurant,Pizza Place,Lounge,Bank,Clothing Store,Supermarket,Metro Station,Moving Target,Caribbean Restaurant,Spanish Restaurant
11,Sunnyside,2,Pizza Place,Chinese Restaurant,Bakery,South American Restaurant,Deli / Bodega,Coffee Shop,Discount Store,Italian Restaurant,Latin American Restaurant,Grocery Store
16,Rego Park,2,Bakery,Pharmacy,Grocery Store,Sushi Restaurant,Sandwich Place,Mediterranean Restaurant,Chinese Restaurant,Pizza Place,Liquor Store,Furniture / Home Store


In [58]:
queens_cluster_3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Floral Park,3,Indian Restaurant,Hobby Shop,Grocery Store,Pizza Place,Salon / Barbershop,Dosa Place,Basketball Court,Falafel Restaurant,Farm,Farmers Market


In [59]:
queens_cluster_4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Utopia,4,Deli / Bodega,Automotive Shop,Playground,Pizza Place,Rental Car Location,Donut Shop,South American Restaurant,Spa,Grocery Store,Basketball Court
